In [1]:
from py2neo import Graph, Node, Relationship, authenticate

In [2]:
authenticate("localhost", "neo4j", "neo4j_train")

In [3]:
g = Graph(host='localhost')

In [4]:
type(g)

py2neo.database.Graph

In [5]:
!ls *.txt

edge_distance.txt node_city.txt


In [6]:
!head node_city.txt

New_York, 8491079
Los_Angeles, 3928864
Chicago, 2722389
Houston, 2239558
Philadelphia, 1560297
Phoenix, 1537058
San_Antonio, 1436697
San_Diego, 1381069
Dallas, 1281047
San_Jose, 1015785


In [7]:
tx = g.begin()
with open('node_city.txt') as f_in:
    for line in f_in:
        city_list = line.rstrip().split(',')
        city = Node("City",name=city_list[0], population=city_list[1])
        tx.create(city)
tx.commit()

In [8]:
g.data("MATCH (c:City) RETURN c.name, c.population")

[{u'c.name': u'New_York', u'c.population': 8491079},
 {u'c.name': u'Los_Angeles', u'c.population': 3928864},
 {u'c.name': u'Chicago', u'c.population': 2722389},
 {u'c.name': u'Houston', u'c.population': 2239558},
 {u'c.name': u'Philadelphia', u'c.population': 1560297},
 {u'c.name': u'Phoenix', u'c.population': 1537058},
 {u'c.name': u'San_Antonio', u'c.population': 1436697},
 {u'c.name': u'San_Diego', u'c.population': 1381069},
 {u'c.name': u'Dallas', u'c.population': 1281047},
 {u'c.name': u'San_Jose', u'c.population': 1015785},
 {u'c.name': u'Austin', u'c.population': 912791},
 {u'c.name': u'Jacksonville', u'c.population': 853382},
 {u'c.name': u'San_Francisco', u'c.population': 852469},
 {u'c.name': u'Indianapolis', u'c.population': 848788},
 {u'c.name': u'Columbus', u'c.population': 835957},
 {u'c.name': u'Fort_Worth', u'c.population': 812238},
 {u'c.name': u'Charlotte', u'c.population': 809958},
 {u'c.name': u'Detroit', u'c.population': 680250},
 {u'c.name': u'Seattle', u'c.popula

In [9]:
g.data("MATCH (c:City) RETURN c.name, c.population LIMIT 10")

[{u'c.name': u'New_York', u'c.population': 8491079},
 {u'c.name': u'Los_Angeles', u'c.population': 3928864},
 {u'c.name': u'Chicago', u'c.population': 2722389},
 {u'c.name': u'Houston', u'c.population': 2239558},
 {u'c.name': u'Philadelphia', u'c.population': 1560297},
 {u'c.name': u'Phoenix', u'c.population': 1537058},
 {u'c.name': u'San_Antonio', u'c.population': 1436697},
 {u'c.name': u'San_Diego', u'c.population': 1381069},
 {u'c.name': u'Dallas', u'c.population': 1281047},
 {u'c.name': u'San_Jose', u'c.population': 1015785}]

In [10]:
result_set = g.node_selector.select("City")

In [11]:
type(result_set)

py2neo.database.selection.NodeSelection

In [12]:
result_set.first()

(f6273ab:City {name:"New_York",population:8491079})

In [13]:
LA = result_set.where(name="Los_Angeles")

In [14]:
type(LA)

py2neo.database.selection.NodeSelection

In [15]:
LA.first()

(b56bd7c:City {name:"Los_Angeles",population:3928864})

In [17]:
for r in result_set:
    print r

(f6273ab:City {name:"New_York",population:8491079})
(b56bd7c:City {name:"Los_Angeles",population:3928864})
(a2a3157:City {name:"Chicago",population:2722389})
(baf1e9f:City {name:"Houston",population:2239558})
(b4826a2:City {name:"Philadelphia",population:1560297})
(eff19f8:City {name:"Phoenix",population:1537058})
(c14fbab:City {name:"San_Antonio",population:1436697})
(d2495e7:City {name:"San_Diego",population:1381069})
(a9c4f52:City {name:"Dallas",population:1281047})
(bd51ebb:City {name:"San_Jose",population:1015785})
(f4d6282:City {name:"Austin",population:912791})
(fad8c11:City {name:"Jacksonville",population:853382})
(a864834:City {name:"San_Francisco",population:852469})
(e701086:City {name:"Indianapolis",population:848788})
(bb883a5:City {name:"Columbus",population:835957})
(e64db45:City {name:"Fort_Worth",population:812238})
(fd41b55:City {name:"Charlotte",population:809958})
(a40bc67:City {name:"Detroit",population:680250})
(b5894f2:City {name:"Seattle",population:66342})
(ce8

*New Steps Begin Here*

In [24]:
!ls *.txt

edge_distance.txt node_city.txt


In [25]:
!head edge_distance.txt

New_York,Los_Angeles, 2451
New_York,Chicago, 733
New_York,Philadelphia, 95
New_York,Dallas, 1373
New_York,San_Francisco, 2572
Los_Angeles,San_Francisco, 382
Los_Angeles,Chicago, 1756
Los_Angeles,Austin, 1377
Los_Angeles,San_Jose, 306
Los_Angeles,Charlotte, 2125


In [29]:
tx = g.begin()
with open('edge_distance.txt') as f_in:
    for line in f_in:
        edge_list = line.rstrip().split(',')
        city1_name = edge_list[0]
        city2_name = edge_list[1]
        city1_node = result_set.where(name=city1_name)
        city2_node = result_set.where(name=city2_name)
        city_pair = Relationship(city1_node, "FLIGHT_BETWEEN", city2_node)
        city_pair["distance"] = edge_list[2]
        tx.create(city_pair)
tx.commit()

In [30]:
g.relationship_types

frozenset({u'FLIGHT_BETWEEN'})

In [31]:
import pandas as pd

In [32]:
flight_list = list()

In [33]:
for cities in g.match(rel_type="FLIGHT_BETWEEN"):
    city1_name = cities.nodes()[0]['name']
    city2_name = cities.nodes()[1]['name']
    distance = cities['distance']
    flight_list.append([city1_name, city2_name, distance])

In [34]:
type(flight_list)

list

In [35]:
flight_list[0]

[u'New_York', u'Los_Angeles', u' 2451']

In [36]:
flight_df = pd.DataFrame(flight_list)

In [37]:
flight_df

,0,1,2
0,New_York,Los_Angeles,2451
1,New_York,Chicago,733
2,New_York,Philadelphia,95
3,New_York,Dallas,1373
4,New_York,San_Francisco,2572
5,Los_Angeles,San_Francisco,382
6,Los_Angeles,Chicago,1756
7,Los_Angeles,Austin,1377
8,Los_Angeles,San_Jose,306
9,Los_Angeles,Charlotte,2125
